In [1]:
import numpy as np
from collections import namedtuple
import time
import random

In [2]:
class Agent:
    def __init__(self, i=0,j=0):
        self.i=i
        self.j=j
    @property
    def loc(self):
        return(self.i,self.j)
    
    def vmove(self,direction):
        direction=1 if direction>0 else -1
        return Agent(self.i + direction, self.j)
    
    def hmove(self,direction):
        direction=1 if direction>0 else -1
        return Agent(self.i , direction+ self.j) 
    def __repr__(self):
        return str(self.loc)

In [3]:
# general maze class defination
class Maze:
   
    # init method or constructor
    
    # defining a general 4*4 matrix intialised as 0 matrix
    def __init__(self, rows=4,columns=4):
        self.env =np.zeros((rows, columns))
        
        # mousy is object of Agent class
        self.mousy=Agent(0,0)
     
    def reset(self):
        self.mousy.i = 0
        self.mousy.j = 0

    def state_for_agent(self,a):
        nr,nc=self.env.shape
        return a.i*nc+a.j
        
    def in_bounds(self,i,j):
        nr,nc= self.env.shape
        return i>=0 and i<nr and j>=0 and j<nc
    
    def agent_in_bounds(self,a):
        return self.in_bounds(a.i,a.j)
    
    def agent_dient(self,a):
        return not self.env[a.i,a.j]== -1
    
    def is_valid_new_agent(self,a):
        return self.agent_in_bounds(a) and self.agent_dient(a)
    
    @property
    def all_actions(self):        
        a=self.mousy    
        return [
            a.vmove(1),
            a.vmove(-1),
            a.hmove(1),
            a.hmove(-1),
        ]
       
    def compute_possible_moves(self):
        moves = self.all_actions
        return [(m, ii) for ii,m in enumerate(moves) if self.is_valid_new_agent(m) ]
    
    def do_a_move(self,a):
        assert self.is_valid_new_agent(a),"Mousy can't go there"
        self.mousy=a
        return 10 if self.has_won() else -0.1
        
    def has_won(self):
        a=self.mousy
        return self.env[a.i,a.j]==1
        
    def visualise(self):
        assert self.agent_in_bounds(self.mousy), "mousy is out of bounds"
        ec=self.env.copy()
        mo=self.mousy
        ec[mo.i,mo.j]=6
        print(ec)
        

In [4]:
class Qlearning:
    def __init__(self, num_states, num_actions, lr=0.1, discount_factor =0.99):
        self.q = np.zeros((num_states, num_actions))
        self.a = lr
        self.g = discount_factor
    
    def update(self, st, at, rt, st1):
        q = self.q  
        a = self.a
        g = self.g
        q[st, at] = (1-a) * q[st, at] + a * (rt + g * np.max(q[st1]))

In [5]:
# A test maze we are trying to solve
def make_test_maze(s=4): 
    m = Maze(s,s)
    e = m.env
    h, w = e.shape
    e[-1,-1]= 1

    for i in range(len(e)):
        for j in range(len(e[i])):
            if i in [0, h-1] and j in [0, w-1]: 
                continue
            if random.random()<0.3:
                e[i,j]= -1
    return m

In [6]:
def main():
    s=5
    q = Qlearning(s**2,4)
    
    go_ahead = False
    
    while not go_ahead:
        M=make_test_maze(s)
        M.visualise()
        ctu = input ()
        if ctu.lower() == 'n':
            continue
        go_ahead=True
    
    for i in range(1000):
        M.reset()  
        print(i)
        final_score=0
        while not M.has_won():
            moves= M.compute_possible_moves()
            random.shuffle(moves)
            move, move_idx = moves[0]
            
            at = move_idx
            st = M.state_for_agent(M.mousy)
            score = M.do_a_move(move)
            final_score += score 
            rt = score
            st1 = M.state_for_agent(M.mousy)

            q.update(st, at , rt, st1)
            

            

        print("final score=",final_score) 
    print(q.q)
                
    M.reset()
    M.visualise()
    
    while not M.has_won():
        s= M.state_for_agent(M.mousy)
        a_idx = np.argmax(q.q[s])
        M.do_a_move(M.all_actions[a_idx])
        M.visualise()
        
    M.visualise()


In [7]:
if __name__=="__main__":
    main()  


[[ 6. -1.  0. -1.  0.]
 [ 0.  0.  0. -1. -1.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0. -1.  0. -1.  1.]]
v
0
final score= 0.700000000000017
1
final score= 6.099999999999998
2
final score= -4.699999999999964
3
final score= 5.300000000000001
4
final score= -19.300000000000146
5
final score= -4.4999999999999645
6
final score= 6.699999999999998
7
final score= 2.9000000000000092
8
final score= 3.300000000000008
9
final score= -1.4999999999999751
10
final score= 5.5
11
final score= 6.299999999999998
12
final score= 9.1
13
final score= 7.499999999999999
14
final score= 4.5000000000000036
15
final score= -5.499999999999961
16
final score= 7.499999999999999
17
final score= 1.7000000000000135
18
final score= 4.300000000000004
19
final score= -3.6999999999999673
20
final score= 8.7
21
final score= 5.699999999999999
22
final score= 6.499999999999998
23
final score= -12.50000000000005
24
final score= 4.900000000000002
25
final score= 8.1
26
final score= -10.900000000000027
27
final score

final score= -14.900000000000084
304
final score= 2.70000000000001
305
final score= 7.699999999999999
306
final score= 7.699999999999999
307
final score= -10.500000000000021
308
final score= 4.100000000000005
309
final score= -32.500000000000334
310
final score= -2.4999999999999716
311
final score= -0.0999999999999801
312
final score= -5.099999999999962
313
final score= 6.699999999999998
314
final score= 7.8999999999999995
315
final score= 7.099999999999999
316
final score= 5.699999999999999
317
final score= -0.2999999999999794
318
final score= -0.4999999999999787
319
final score= -7.299999999999976
320
final score= -13.300000000000061
321
final score= 6.299999999999998
322
final score= -5.69999999999996
323
final score= 4.100000000000005
324
final score= 0.5000000000000178
325
final score= 0.10000000000001918
326
final score= 7.099999999999999
327
final score= 3.500000000000007
328
final score= 6.099999999999998
329
final score= -1.6999999999999744
330
final score= 4.300000000000004
3

final score= -1.0999999999999766
540
final score= 8.5
541
final score= 6.299999999999998
542
final score= 6.499999999999998
543
final score= -6.699999999999967
544
final score= 4.5000000000000036
545
final score= 4.900000000000002
546
final score= 1.1000000000000156
547
final score= 2.3000000000000114
548
final score= 5.5
549
final score= 2.5000000000000107
550
final score= -19.50000000000015
551
final score= -37.70000000000041
552
final score= 8.1
553
final score= 7.099999999999999
554
final score= -40.90000000000045
555
final score= -7.299999999999976
556
final score= -14.700000000000081
557
final score= 1.1000000000000156
558
final score= 1.7000000000000135
559
final score= -15.700000000000095
560
final score= -22.500000000000192
561
final score= 7.099999999999999
562
final score= 7.099999999999999
563
final score= 2.3000000000000114
564
final score= -20.30000000000016
565
final score= 0.700000000000017
566
final score= 7.699999999999999
567
final score= -16.900000000000112
568
fina

final score= -20.30000000000016
840
final score= 8.299999999999999
841
final score= 7.299999999999999
842
final score= 1.9000000000000128
843
final score= 0.3000000000000185
844
final score= 5.899999999999999
845
final score= 4.700000000000003
846
final score= 4.300000000000004
847
final score= 6.499999999999998
848
final score= 2.3000000000000114
849
final score= -0.8999999999999773
850
final score= 3.7000000000000064
851
final score= 7.299999999999999
852
final score= -22.700000000000195
853
final score= 8.5
854
final score= 6.499999999999998
855
final score= -0.0999999999999801
856
final score= 1.9000000000000128
857
final score= -8.499999999999993
858
final score= -6.099999999999959
859
final score= 7.8999999999999995
860
final score= -8.699999999999996
861
final score= 4.700000000000003
862
final score= -6.099999999999959
863
final score= -1.6999999999999744
864
final score= -34.700000000000365
865
final score= 6.899999999999999
866
final score= 2.9000000000000092
867
final score=